In [18]:
import tensorflow as tf
import numpy as np
import os
from tensorflow.python.client import device_lib
import pandas as pd
from tqdm.auto import tqdm
from PIL import Image
size = 224
os.environ["CUDA_VISIBLE_DEVICES"] = "0"


In [19]:
def keras2TFlite(model_path,name):
    #load a pre-trained model
    with tf.device('/gpu:0'):
        model = tf.keras.models.load_model(model_path)
        #convert the model
        converter = tf.lite.TFLiteConverter.from_keras_model(model)
        tflite_model = converter.convert()
        #save the converted model
        with open('../../model/classification/tflite/'+name+'.tflite', 'wb') as f:
            f.write(tflite_model)
        print('Model converted successfully')

def createDirectory(directory):
    """_summary_
        create Directory
    Args:
        directory (string): file_path
    """    
    try:
        if not os.path.exists(directory):
            os.makedirs(directory)
    except OSError:
        print("Error: Failed to create the directory.")
# keras2TFlite('../../model/classification/MobileNetV2_checkpoints.h5','MobileNetV2')
# keras2TFlite('../../model/classification/MobileNetV1_acc_checkpoints.h5','MobileNetV1')
# keras2TFlite('../../model/classification/DenseNet121_checkpoints.h5','DenseNet121')
# keras2TFlite('../../model/classification/EfficientNetV2B0_checkpoints.h5','EfficientNetV2B0')
# keras2TFlite('../../model/classification/NASNetMobile_checkpoints.h5','NASNetMobile')
# keras2TFlite('../../model/classification/ResNet50_checkpoints.h5','ResNet50')        

In [20]:

test_df = pd.read_csv(
    '../../data/classificationDDH/test_aug_classification_dataset.csv')


test_img_list = test_df['file name'].to_list()
test_label_list = test_df['standard class'].to_list()
test_case_list = test_df['case'].to_list()
test_img_path = '../../data/classificationDDH/aug_dataset/test/'


x_test = np.zeros((len(test_img_list), size, size, 3), dtype=np.uint8)
for i in tqdm(range(len(test_img_list))):
    x_test[i] = np.array(Image.open(
        test_img_path+str(test_case_list[i])+'/'+test_img_list[i]).resize((size, size)))
y_test = np.array(test_label_list)

  0%|          | 0/3476 [00:00<?, ?it/s]